# 使用AI解译模型进行交互式SAR水体提取
用户可使用平台公开影像或者个人上传影像在开发者模式下使用SDK进行交互式的在线模型推理，推理之后的返回值为aie.Image。

## 初始化环境

In [1]:
import aie

aie.Authenticate()
aie.Initialize()

## 数据集检索
使用 FeatureCollection 引用平台内置或个人上传的矢量边界，定义检索数据的区域。设置检索的空间和时间范围，以及属性过滤条件，对Sentinel-1 SAR GRD数据集进行检索。

In [2]:
# 指定需要检索的区域
feature_collection = aie.FeatureCollection('China_Province') \
                        .filter(aie.Filter.eq('province', '浙江省'))

geometry = feature_collection.geometry()

# 指定检索数据集，可设置检索的空间和时间范围，以及属性过滤条件（如传感器成像模式等）
image = aie.ImageCollection('SENTINEL_GRD') \
             .filterBounds(geometry) \
             .filterDate('2021-01-01', '2021-12-31') \
             .filter(aie.Filter.eq('sar:instrument_mode', 'IW')) \
             .limit(100).mosaic()

## 使用AI解译模型进行在线交互式推理
构造AI推理模型，对检索的影像数据选取波段进行模型推理。

In [3]:
# 进行sar水体模型推理
model = aie.Model('sar_water')
predict_image = model.predict(image.select(['VV']))

## 结果可视化
对检索出的影像数据和推理结果添加图层，进行可视化。**不同影像和分辨率下的模型效果可能有所差别，建议对影像调整到一个合适的地图层级zoom，水平拖动进行在线交互式结果查看，尽量避免图层缩放影响模型解译效果**。

In [4]:
map_view_bounds = image.getBounds()
map_view_center = image.getCenter()

map = aie.Map(
    center=map_view_center,
    height=800,
    zoom=7
)

vis_params = {
    'bands': ['VV'],
    'min': 30,
    'max': 350,
}
predict_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['#FFFFFF','#F8D072']
}

map.addLayer(
    image,
    vis_params,
    'VV',
    bounds=map_view_bounds
)
map.addLayer(
    predict_image,
    predict_vis_params,
    'predict',
    bounds=map_view_bounds
)
map

Map(center=[30.643262768216722, 120.84707528495156], controls=(ZoomControl(options=['position', 'zoom_in_text'…